<a href="https://colab.research.google.com/github/blankon123/random-analytics/blob/main/Scikit_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Libriaries

In [1]:
from sklearn import datasets
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd

## Download Kaggle Data

In [2]:
#Download dataset dari kaggle
from google.colab import files
from IPython.display import clear_output

files.upload()

! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c bigdata-bps-hackathon-2021

clear_output()
!ls /content

Deskripsi%20Data%20TPK.docx  test-online_booking_2021.csv
kaggle.json		     test-TPK_Hotel_berbintang_2021.csv
sample_data		     train-online_booking_2020.csv
sample-submission.csv	     train-TPK_Hotel_berbintang_2020.csv


## Data Loading

### Training Data

In [3]:
train_online = pd.read_csv('train-online_booking_2020.csv')
train_online.head(10)

,province,type,tanggal,klasifikasi,all_available_room,room_total
0,Bali,Hotel,1/1/2020,Rated,2155,37756
1,Bali,Hotel,1/1/2020,Not Rated,163,392
2,Bali,Hotel,1/2/2020,Rated,3878,38814
3,Bali,Hotel,1/2/2020,Not Rated,211,415
4,Bali,Hotel,1/3/2020,Rated,5412,39084
5,Bali,Hotel,1/3/2020,Not Rated,256,444
6,Bali,Hotel,1/4/2020,Rated,7786,39310
7,Bali,Hotel,1/4/2020,Not Rated,306,483
8,Bali,Hotel,1/5/2020,Rated,7956,39334
9,Bali,Hotel,1/5/2020,Not Rated,273,448


In [4]:
train_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   province            696 non-null    object
 1   type                696 non-null    object
 2   tanggal             696 non-null    object
 3   klasifikasi         696 non-null    object
 4   all_available_room  696 non-null    int64 
 5   room_total          696 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 32.8+ KB


In [5]:
train_bps = pd.read_csv('train-TPK_Hotel_berbintang_2020.csv')
train_bps.head()

,Id,Provinsi,Tahun,Bulan,Aggregate_var,TPK
0,1,Bali,2020,Januari,NaN,59.29
1,2,Bali,2020,Februari,NaN,45.98
2,3,Bali,2020,Maret,NaN,25.41
3,4,Bali,2020,April,NaN,3.22
4,5,Bali,2020,Mei,NaN,2.07


What is `'Aggregate_var'` ?

In [6]:
train_bps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             12 non-null     int64  
 1   Provinsi       12 non-null     object 
 2   Tahun          12 non-null     int64  
 3   Bulan          12 non-null     object 
 4   Aggregate_var  0 non-null      float64
 5   TPK            12 non-null     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 704.0+ bytes


### Testing Data

In [7]:
test_bps = pd.read_csv('test-TPK_Hotel_berbintang_2021.csv')
test_bps

,Id,Provinsi,Tahun,Bulan,Aggregate,TPK
0,1,BALI,2021,Januari,NaN,NaN
1,2,BALI,2021,Februari,NaN,NaN
2,3,BALI,2021,Maret,NaN,NaN
3,4,BALI,2021,April,NaN,NaN
4,5,BALI,2021,Mei,NaN,NaN
5,6,BALI,2021,Juni,NaN,NaN


What is `'Aggregate'` ?

In [8]:
test_online = pd.read_csv('test-online_booking_2021.csv')
test_online.head()

,province,type,tanggal,klasifikasi,all_available_room,room_total
0,Bali,Hotel,1/1/2021,Rated,31334,54852
1,Bali,Hotel,1/1/2021,Not Rated,618,702
2,Bali,Hotel,1/2/2021,Rated,33698,55950
3,Bali,Hotel,1/2/2021,Not Rated,642,705
4,Bali,Hotel,1/3/2021,Rated,33310,55380


In [9]:
test_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   province            356 non-null    object
 1   type                356 non-null    object
 2   tanggal             356 non-null    object
 3   klasifikasi         356 non-null    object
 4   all_available_room  356 non-null    int64 
 5   room_total          356 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 16.8+ KB


### Submission

In [10]:
submission = pd.read_csv('sample-submission.csv')
submission

,Id,TPK
0,1,14.55
1,2,7.99
2,3,8.56
3,4,9.01
4,5,8.65
5,6,18.56


## Preprocessing

### Calculating Daily Used Room

In [11]:
train_online['used_room'] = train_online.room_total-train_online.all_available_room
test_online['used_room']  = test_online.room_total - test_online.all_available_room

### Agregating to Monthly Basis

First we need to cast field 'tanggal' to datetime type

In [12]:
train_online['tanggal'] =  pd.to_datetime(train_online['tanggal'])
test_online['tanggal']  =  pd.to_datetime(test_online['tanggal'])

Now we can agregate them

In [13]:
train_online_agg = train_online \
                    .set_index(['tanggal']) \
                    .groupby(pd.Grouper(freq='M')) \
                    .sum()
test_online_agg  = test_online \
                    .set_index(['tanggal']) \
                    .groupby(pd.Grouper(freq='M')) \
                    .sum()

In [14]:
train_online_agg.head()

,all_available_room,room_total,used_room
tanggal,,,
2020-01-31,326629,1303572,976943
2020-02-29,353685,1093452,739767
2020-03-31,954093,1674561,720468
2020-04-30,982067,1318197,336130
2020-05-31,980395,1494303,513908


In [15]:
test_online_agg.head()

,all_available_room,room_total,used_room
tanggal,,,
2021-01-31,1076383,1732399,656016
2021-02-28,870157,1466726,596569
2021-03-31,985358,1562210,576852
2021-04-30,1021319,1624712,603393
2021-05-31,996777,1588513,591736


### Calculating Monthly Occupancy Rate

In [16]:
train_online_agg['TPK_Online'] = train_online_agg['used_room']/train_online_agg['room_total']*100
test_online_agg['TPK_Online']  = test_online_agg['used_room']/test_online_agg['room_total']*100

In [17]:
train_online_agg.head()

,all_available_room,room_total,used_room,TPK_Online
tanggal,,,,
2020-01-31,326629,1303572,976943,74.943540
2020-02-29,353685,1093452,739767,67.654273
2020-03-31,954093,1674561,720468,43.024291
2020-04-30,982067,1318197,336130,25.499224
2020-05-31,980395,1494303,513908,34.391151


In [18]:
test_online_agg.head()

,all_available_room,room_total,used_room,TPK_Online
tanggal,,,,
2021-01-31,1076383,1732399,656016,37.867489
2021-02-28,870157,1466726,596569,40.673514
2021-03-31,985358,1562210,576852,36.925381
2021-04-30,1021319,1624712,603393,37.138459
2021-05-31,996777,1588513,591736,37.250938


### Join Datasets

First we need to reset the index and make 'Id' column for join purpose

In [19]:
train_online_agg = train_online_agg.reset_index()
test_online_agg  = test_online_agg.reset_index()

In [20]:
train_online_agg['Id'] = pd. DatetimeIndex(train_online_agg['tanggal']).month
test_online_agg['Id']  = pd. DatetimeIndex(test_online_agg['tanggal']).month

Once both the data have 'Id' column, we can join them based on that column

In [21]:
train_full = pd.merge(train_online_agg,train_bps,on='Id',how='left')
train_full.head()

,tanggal,all_available_room,room_total,used_room,TPK_Online,Id,Provinsi,Tahun,Bulan,Aggregate_var,TPK
0,2020-01-31,326629,1303572,976943,74.943540,1,Bali,2020,Januari,NaN,59.29
1,2020-02-29,353685,1093452,739767,67.654273,2,Bali,2020,Februari,NaN,45.98
2,2020-03-31,954093,1674561,720468,43.024291,3,Bali,2020,Maret,NaN,25.41
3,2020-04-30,982067,1318197,336130,25.499224,4,Bali,2020,April,NaN,3.22
4,2020-05-31,980395,1494303,513908,34.391151,5,Bali,2020,Mei,NaN,2.07


In [22]:
test_full = pd.merge(test_online_agg,test_bps,on='Id',how='left')
test_full.head()

,tanggal,all_available_room,room_total,used_room,TPK_Online,Id,Provinsi,Tahun,Bulan,Aggregate,TPK
0,2021-01-31,1076383,1732399,656016,37.867489,1,BALI,2021,Januari,NaN,NaN
1,2021-02-28,870157,1466726,596569,40.673514,2,BALI,2021,Februari,NaN,NaN
2,2021-03-31,985358,1562210,576852,36.925381,3,BALI,2021,Maret,NaN,NaN
3,2021-04-30,1021319,1624712,603393,37.138459,4,BALI,2021,April,NaN,NaN
4,2021-05-31,996777,1588513,591736,37.250938,5,BALI,2021,Mei,NaN,NaN


As we saw above, we need to select only informative variable in our dataset

In [23]:
trainset = train_full[['TPK_Online','TPK']]
testset  = test_full[['TPK_Online','TPK']]

In [24]:
trainset

,TPK_Online,TPK
0,74.943540,59.29
1,67.654273,45.98
2,43.024291,25.41
3,25.499224,3.22
4,34.391151,2.07
5,32.828614,2.07
6,31.544996,2.57
7,37.623642,3.68
8,39.291089,5.28
9,37.940120,9.53


In [25]:
testset

,TPK_Online,TPK
0,37.867489,NaN
1,40.673514,NaN
2,36.925381,NaN
3,37.138459,NaN
4,37.250938,NaN
5,38.742409,NaN


## Transformation

### Minmax Scaler

In [26]:
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

minmax_scaler_tpk = MinMaxScaler()
trainset['TPK_Minmax'] = minmax_scaler_tpk\
                        .fit_transform(trainset['TPK'].values.reshape(-1,1))

minmax_scaler_tpk_online = MinMaxScaler()
trainset['TPK_O_Minmax'] = minmax_scaler_tpk_online\
                           .fit_transform(trainset['TPK_Online'].values.reshape(-1,1))
testset['TPK_O_Minmax'] = minmax_scaler_tpk_online\
                            .transform(testset['TPK_Online'].values.reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [27]:
trainset.head()

,TPK_Online,TPK,TPK_Minmax,TPK_O_Minmax
0,74.943540,59.29,1.000000,1.000000
1,67.654273,45.98,0.767389,0.852576
2,43.024291,25.41,0.407899,0.354440
3,25.499224,3.22,0.020098,0.000000
4,34.391151,2.07,0.000000,0.179837


In [28]:
testset.head()

,TPK_Online,TPK,TPK_O_Minmax
0,37.867489,NaN,0.250145
1,40.673514,NaN,0.306897
2,36.925381,NaN,0.231091
3,37.138459,NaN,0.235401
4,37.250938,NaN,0.237676


## Exporting to CSV

In [29]:
trainset.to_csv('TRAINSET.csv')

In [30]:
testset.to_csv('TESTSET.csv')

In [31]:
!ls /content

Deskripsi%20Data%20TPK.docx   TESTSET.csv
kaggle.json		      test-TPK_Hotel_berbintang_2021.csv
sample_data		      train-online_booking_2020.csv
sample-submission.csv	      TRAINSET.csv
test-online_booking_2021.csv  train-TPK_Hotel_berbintang_2020.csv
